In [1]:
# === CELL 1 (v23b - + Argument Generation Loss) ===
import re
import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, pipeline
from peft import LoraConfig, get_peft_model, TaskType
from random import sample, random
from transformers import AutoTokenizer as AutoTokenizerNLI, AutoModelForSequenceClassification

# Load tokenizer and paraphraser globally
tokenizer = T5Tokenizer.from_pretrained("t5-base")
paraphraser = pipeline("text2text-generation", model="ramsrigouthamg/t5_paraphraser")

# Load NLI model
nli_tokenizer = AutoTokenizerNLI.from_pretrained("facebook/bart-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli").eval()

@torch.no_grad()
def nli_contradiction_loss(premises, hypotheses):
    losses = []
    for premise, hypo in zip(premises, hypotheses):
        inputs = nli_tokenizer(premise, hypo, return_tensors="pt", truncation=True, padding=True)
        outputs = nli_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        contradiction_prob = probs[:, 2]  # label 2 = contradiction
        loss = 1.0 - contradiction_prob.mean()
        losses.append(loss)
    return torch.stack(losses).mean()

@torch.no_grad()
def topic_relevance_loss(topics, generations):
    losses = []
    for topic, gen in zip(topics, generations):
        inputs = nli_tokenizer(topic, gen, return_tensors="pt", truncation=True, padding=True)
        outputs = nli_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        entail_prob = probs[:, 0]  # label 0 = entailment
        loss = 1.0 - entail_prob.mean()
        losses.append(loss)
    return torch.stack(losses).mean()

@torch.no_grad()
def semantic_similarity_loss(refs, hypos):
    losses = []
    for r, h in zip(refs, hypos):
        inputs = nli_tokenizer(r, h, return_tensors="pt", truncation=True, padding=True)
        outputs = nli_model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        similarity_prob = probs[:, 0]  # entailment
        losses.append(similarity_prob.mean())
    return 1.0 - torch.stack(losses).mean()

# === New: Argument generation detection using simple keyword pattern ===
def argument_presence_loss(paragraphs):
    keywords = ["because", "as a result", "due to", "this means", "this is because", "for example", "for instance"]
    losses = []
    for para in paragraphs:
        score = any(k in para.lower() for k in keywords)
        loss = 0.0 if score else 1.0
        losses.append(torch.tensor(loss))
    return torch.stack(losses).mean()

def lexical_diversity_loss(labels, pad_token_id=0):
    losses = []
    for seq in labels:
        words = [t for t in seq if t != pad_token_id]
        unique = len(set(words))
        total = len(words)
        penalty = 1.0 - unique / total if total > 0 else 0.0
        losses.append(torch.tensor(penalty, device=labels.device))
    return torch.stack(losses).mean()

def repetition_overlap_loss(body1s, body2s):
    losses = []
    for b1, b2 in zip(body1s, body2s):
        set1 = set(b1.lower().split())
        set2 = set(b2.lower().split())
        overlap = len(set1 & set2) / max(1, len(set2))
        losses.append(torch.tensor(overlap))
    return torch.stack(losses).mean()

def ngram_overlap_loss(sequences, n=3):
    losses = []
    for seq in sequences:
        tokens = seq.lower().split()
        ngrams = set(tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1))
        losses.append(torch.tensor(1.0 - len(ngrams) / max(1, len(tokens)), device='cpu'))
    return torch.stack(losses).mean()

def argument_distance_loss(body1s, body2s):
    return semantic_similarity_loss(body1s, body2s)  # Higher similarity → higher loss

def dynamic_mask_input(text, tokenizer, mask_rate=0.15):
    tokens = tokenizer.tokenize(text)
    if len(tokens) < 4:
        return text
    num_to_mask = max(1, int(len(tokens) * mask_rate))
    for i in sample(range(len(tokens)), num_to_mask):
        tokens[i] = "<extra_id_0>"
    return tokenizer.convert_tokens_to_string(tokens)

def t5_paraphrase_text(text):
    result = paraphraser(f"paraphrase: {text} </s>", max_length=128, num_return_sequences=1, do_sample=True)
    return result[0]["generated_text"] if result else text

# === Load and prepare dataset ===
raw_dataset = load_dataset("chillies/IELTS-writing-task-2-evaluation", split="train")

def is_valid(example):
    try:
        band = float(re.sub(r"[^\d.]", "", example["band"]))
        return band >= 7.0 and example["essay"] and len(example["essay"].split()) > 220
    except:
        return False

filtered = [ex for ex in raw_dataset if is_valid(ex)]

def split_paragraphs_flex(essay):
    paras = [p.strip() for p in re.split(r"\n{2,}", essay.strip()) if p.strip()]
    return paras[0], paras[1], paras[2], paras[-1] if len(paras) >= 4 else None

split_data = []
for ex in filtered:
    try:
        result = split_paragraphs_flex(ex["essay"])
        if result is None:
            continue
        intro, body1, body2, conclusion = result
        if all(len(p.split()) > t for p, t in zip([intro, body1, body2, conclusion], [40, 60, 70, 35])) and ex["prompt"][:30] not in intro:
            set1, set2 = set(body1.lower().split()), set(body2.lower().split())
            if len(set1 & set2) / max(1, len(set2)) < 0.7:
                split_data.append({
                    "prompt": ex["prompt"].strip(),
                    "intro": intro.strip(),
                    "body1": body1.strip(),
                    "body2": body2.strip(),
                    "conclusion": conclusion.strip()
                })
    except:
        continue

print("\n📊 Filtered Samples:", len(split_data))

# === Define train function ===
def train_paragraph_model(field, save_dir, max_target_length=256):
    print(f"\n🚀 Training for: {field.upper()}", flush=True)
    data = []
    for ex in split_data:
        if len(ex["prompt"]) < 10 or len(ex[field]) < 30:
            continue
        prompt = dynamic_mask_input(ex["prompt"], tokenizer) if random() < 0.5 else ex["prompt"]
        if field == "intro":
            input_text = f"Write a short and clear INTRODUCTION:\n\n{prompt}\n\n- Paraphrase topic\n- State opinion\n- Brief background"
        elif field == "body1":
            input_text = f"Write the FIRST BODY PARAGRAPH for:\n\n{prompt}\n\n- Clear argument\n- Specific example\n- Logical explanation"
        elif field == "body2":
            intro = dynamic_mask_input(ex["intro"], tokenizer) if random() < 0.5 else ex["intro"]
            body1_masked = dynamic_mask_input(ex["body1"], tokenizer) if random() < 0.3 else ex["body1"]
            topic_masked = dynamic_mask_input(prompt, tokenizer) if random() < 0.3 else prompt
            input_text = (
                f"Write the SECOND BODY PARAGRAPH that presents a CONTRASTING perspective.\n\n"
                f"TOPIC: {topic_masked}\n\nINTRO: {intro}\n\nBODY 1: {body1_masked}\n\n"
                "Requirements:\n- Start with a contrast linker\n- Opposing idea\n- Specific example\n- Avoid repeating Body 1"
            )
        elif field == "conclusion":
            intro = t5_paraphrase_text(ex["intro"]) if random() < 0.6 else ex["intro"]
            input_text = (
                f"Write a CONCLUSION:\n\nTOPIC: {prompt}\n\nINTRO (paraphrased): {intro}\n\n"
                "Instructions:\n- Restate opinion\n- Summarise main points\n- End strongly"
            )
        data.append({
            "input_text": input_text,
            "target_text": ex[field],
            "intro": ex["intro"],
            "body1": ex["body1"],
            "body2": ex["body2"],
            "topic": ex["prompt"]
        })

    dataset = Dataset.from_list(data).train_test_split(test_size=0.1, seed=42)

    def tokenize_fn(batch):
        inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
        targets = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=max_target_length)
        inputs["labels"] = targets["input_ids"]
        if field in ["body2", "conclusion"]:
            intros = tokenizer(batch["intro"], padding="max_length", truncation=True, max_length=256)
            inputs["intro"] = intros["input_ids"]
        if field in ["body1", "body2"]:
            inputs["body1_text"] = batch["body1"]
        if field == "body2":
            inputs["body2_text"] = batch["body2"]
            inputs["topic"] = batch["topic"]
        return inputs

    tokenized = dataset.map(tokenize_fn, batched=True)

    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    lora = LoraConfig(r=32, lora_alpha=64, target_modules=["q", "v"], lora_dropout=0.05, bias="none", task_type=TaskType.SEQ_2_SEQ_LM)
    model = get_peft_model(model, lora)

    # === Inside dpo_loss: add argument_presence_loss ===
    # === Inside dpo_loss: add argument_presence_loss for body1 and body2 ===
    def dpo_loss(logits, labels, intros=None, body1_text=None, body2_text=None, topic_text=None, pad_token_id=0):
        logits = logits.view(-1, logits.size(-1))
        labels = labels.view(-1)
        mask = labels != pad_token_id
        base = F.cross_entropy(logits[mask], labels[mask]) if mask.any() else torch.tensor(0.0, device=logits.device)
        l_lex = lexical_diversity_loss(labels.view(1, -1), pad_token_id)
        l_contra = nli_contradiction_loss(body1_text, body2_text) if body1_text is not None else 0.0
        l_topic = topic_relevance_loss(topic_text, body2_text) * 0.9 if topic_text is not None else 0.0
        l_rep = repetition_overlap_loss(body1_text, body2_text) if body1_text is not None else 0.0
        l_ngram = ngram_overlap_loss(body2_text) if body2_text is not None else 0.0
        l_arg = argument_distance_loss(body1_text, body2_text) if body1_text is not None else 0.0
        l_sem = semantic_similarity_loss(intros, body2_text) if intros is not None and body2_text is not None else 0.0
        l_gen_b2 = argument_presence_loss(body2_text) if body2_text is not None else 0.0
        l_gen_b1 = argument_presence_loss(body1_text) if body1_text is not None else 0.0
        return base + 0.2 * l_lex + 0.7 * l_contra + 0.9 * l_topic + 0.5 * l_rep + 0.4 * l_ngram + 0.4 * l_arg + 0.4 * l_sem + 0.5 * l_gen_b2 + 0.3 * l_gen_b1

    class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
            labels = inputs.get("labels")
            intros = inputs.get("intro")
            body1_text = inputs.get("body1_text")
            body2_text = inputs.get("body2_text")
            topic_text = inputs.get("topic")
            outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
            loss = dpo_loss(outputs.logits, labels, intros, body1_text, body2_text, topic_text)
            return (loss, outputs) if return_outputs else loss

    args = TrainingArguments(
        output_dir=save_dir,
        num_train_epochs=4,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        learning_rate=3e-4,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir=f"{save_dir}/logs",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_steps=20,
        report_to="none",
        fp16=True,
    )

    trainer = CustomTrainer(
        model=model,
        args=args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"]
    )

    trainer.train()
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"✅ Saved model to: {save_dir}", flush=True)

# === Train all paragraph models for v23 ===
train_paragraph_model("intro", "./t5_intro_lora_v23", max_target_length=160)
train_paragraph_model("body1", "./t5_body1_lora_v23", max_target_length=240)
train_paragraph_model("body2", "./t5_body2_lora_v23", max_target_length=288)
train_paragraph_model("conclusion", "./t5_conclusion_lora_v23", max_target_length=96)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0



📊 Filtered Samples: 419

🚀 Training for: INTRO


Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


...

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

c:\Users\VUONGLOCTRUONG\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,3.155500,2.887333
2,3.110800,2.841069
3,2.986600,2.824035
4,2.899600,2.816853


✅ Saved model to: ./t5_conclusion_lora_v23


In [1]:
# === CELL 2 (v23 - Stronger Eval + Argument Check BODY1/BODY2) ===
import re
import language_tool_python
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# === Load fine-tuned models ===
def load_lora_hf_pipeline(path, base="t5-base", max_length=320):
    base_model = T5ForConditionalGeneration.from_pretrained(base)
    model = PeftModel.from_pretrained(base_model, path)
    tokenizer = T5Tokenizer.from_pretrained(path)
    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.5,
        no_repeat_ngram_size=4
    )
    return HuggingFacePipeline(pipeline=pipe)

llm_intro = load_lora_hf_pipeline("./t5_intro_lora_v23", max_length=160)
llm_body1 = load_lora_hf_pipeline("./t5_body1_lora_v23", max_length=240)
llm_body2 = load_lora_hf_pipeline("./t5_body2_lora_v23", max_length=288)
llm_concl = load_lora_hf_pipeline("./t5_conclusion_lora_v23", max_length=96)

# === Prompt templates ===
prompt_intro = PromptTemplate(
    input_variables=["topic"],
    template=(
        "Write a short and clear INTRODUCTION for this IELTS Writing Task 2 topic:\n\n{topic}\n\n"
        "Requirements:\n"
        "- Paraphrase the topic clearly\n"
        "- State your opinion\n"
        "- Add brief context\n"
        "- Stay strictly on topic. Do NOT introduce unrelated content\n"
        "- Use formal academic tone"
    )
)

prompt_body1 = PromptTemplate(
    input_variables=["topic"],
    template=(
        "Write the FIRST BODY PARAGRAPH for this IELTS Writing Task 2 topic:\n\n{topic}\n\n"
        "Instructions:\n"
        "- Present ONE clear reason to support your opinion\n"
        "- Provide logic and ONE specific example\n"
        "- Avoid vague generalisations and unrelated facts\n"
        "- Keep ideas fully aligned with the topic above"
    )
)

prompt_body2 = PromptTemplate(
    input_variables=["topic", "intro", "body1"],
    template=(
        "Write the SECOND BODY PARAGRAPH for this IELTS Writing Task 2 essay.\n\n"
        "TOPIC: {topic}\n\n"
        "INTRO: {intro}\n\n"
        "BODY 1: {body1}\n\n"
        "Instructions:\n"
        "- Present a CLEAR CONTRASTING viewpoint\n"
        "- Use a new idea and a different example\n"
        "- DO NOT repeat phrases, ideas, or examples from Body 1\n"
        "- Begin with a contrast linker (e.g., 'However', 'On the other hand')\n"
        "- Stay strictly relevant to the given TOPIC"
    )
)

prompt_concl = PromptTemplate(
    input_variables=["topic", "intro"],
    template=(
        "Write a CONCLUSION paragraph for this IELTS Writing Task 2 essay.\n\n"
        "TOPIC: {topic}\n\n"
        "INTRODUCTION: {intro}\n\n"
        "Instructions:\n"
        "- Restate your opinion using DIFFERENT words\n"
        "- Briefly summarise both body paragraphs\n"
        "- End with a strong final sentence (recommendation or reflection)"
    )
)

# === Chains ===
chain_intro = LLMChain(llm=llm_intro, prompt=prompt_intro, output_key="intro")
chain_body1 = LLMChain(llm=llm_body1, prompt=prompt_body1, output_key="body1")
chain_body2 = LLMChain(llm=llm_body2, prompt=prompt_body2, output_key="body2")
chain_concl = LLMChain(llm=llm_concl, prompt=prompt_concl, output_key="conclusion")

full_essay_chain = SequentialChain(
    chains=[chain_intro, chain_body1, chain_body2, chain_concl],
    input_variables=["topic"],
    output_variables=["intro", "body1", "body2", "conclusion"],
    verbose=False,
)

# === Evaluation Function ===
tool = language_tool_python.LanguageTool('en-US')

def evaluate_essay(paragraphs):
    full_text = " ".join(paragraphs)
    word_count = len(full_text.split())
    repeated = [w for w in set(full_text.lower().split()) if full_text.lower().split().count(w) > 2 and len(w) > 3]
    grammar_matches = tool.check(full_text)
    grammar_errors = len(grammar_matches)
    grammar_samples = [match.message for match in grammar_matches[:3]]
    coherence_words = ["first", "second", "furthermore", "however", "in conclusion", "to begin", "on the other hand"]
    coherence_score = 1.0 if any(w in full_text.lower() for w in coherence_words) else 0.5
    intro = paragraphs[0].lower()
    concl = paragraphs[-1].lower()
    conflict = "contradict" if (("agree" in intro and "disagree" in concl) or ("disagree" in intro and "agree" in concl)) else "aligned"
    has_example = any(kw in full_text.lower() for kw in ["for example", "such as", "for instance"])

    body1_set = set(paragraphs[1].lower().split())
    body2_set = set(paragraphs[2].lower().split())
    overlap = len(body1_set & body2_set) / max(1, len(body2_set))
    contradiction = "Yes" if overlap < 0.3 and any(w in paragraphs[2].lower() for w in ["however", "on the other hand", "in contrast"]) else "No"

    # === Argument keyword check for BODY1 and BODY2 ===
    arg_keywords = ["because", "as a result", "due to", "for example", "for instance", "this is because", "consequently"]
    def has_argument_keywords(text):
        return any(k in text.lower() for k in arg_keywords)

    body1_argument = has_argument_keywords(paragraphs[1])
    body2_argument = has_argument_keywords(paragraphs[2])

    return {
        "Word Count": word_count,
        "Repeated Words (>2)": repeated,
        "Grammar Errors": grammar_errors,
        "Grammar Issues (Sample)": grammar_samples,
        "Coherence Score": coherence_score,
        "Intro-Conclusion Logic Conflict": conflict,
        "Has Example": has_example,
        "Body1-Body2 Overlap": round(overlap, 2),
        "Body1-Body2 Contradiction": contradiction,
        "Body1 Has Argument": body1_argument,
        "Body2 Has Argument": body2_argument
    }

# === Generate and evaluate essays ===
def generate_until_min_words(topic, min_words=220, max_tries=5):
    for _ in range(max_tries):
        result = full_essay_chain({"topic": topic})
        paragraphs = [result[k] for k in ["intro", "body1", "body2", "conclusion"]]
        evaluation = evaluate_essay(paragraphs)
        if evaluation["Word Count"] >= min_words and evaluation["Body1-Body2 Overlap"] <= 0.6:
            return paragraphs, evaluation
    return paragraphs, evaluation

# === Topics ===
topics = [
    "Interviews form the basic criteria for most large companies. However, some people think that the interview is not a reliable method of choosing whom to employ and there are other better methods. To what extent do you agree or disagree?",
    "Children find it difficult to concentrate on or pay attention to school. What are the reasons? How can we solve this problem?",
    "Some people think that instead of preventing climate change, we need to find a way to live with it. To what extent do you agree or disagree?",
    "Consumers are faced with increasing numbers of advertisements from competing companies. To what extent do you think are consumers influenced by advertisement? What measures can be taken to protect them?"
]

# === Run and print ===
for i, topic in enumerate(topics, 1):
    print(f"\n{'='*20} 📝 IELTS Essay {i} {'='*20}\n")
    paragraphs, evaluation = generate_until_min_words(topic)
    print("🔹 INTRO:\n", paragraphs[0], "\n")
    print("🟩 BODY 1:\n", paragraphs[1], "\n")
    print("🟨 BODY 2:\n", paragraphs[2], "\n")
    print("🟥 CONCLUSION:\n", paragraphs[3], "\n")
    print("📊 Evaluation:")
    for k, v in evaluation.items():
        print(f"{k}: {v}")


Device set to use cuda:0
The model 'PeftModelForSeq2SeqLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGen


==================== 📝 IELTS Essay 1 ====================



C:\Users\VUONGLOCTRUONG\AppData\Local\Temp\ipykernel_29120\4088496137.py:148: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = full_essay_chain({"topic": topic})


🔹 INTRO:
 Some people have claimed that the interview is not a reliable way of choosing the right person to work for. I agree that there are other methods, which can be used to help the candidate. In my opinion, I believe that the interview should be a more reliable method of selecting someone. 

🟩 BODY 1:
 To begin with, people are accustomed to interviewing and it is said that they do not have the knowledge of the business or their skills. They think that this is because they do not know how to handle the company effectively. Therefore, they do not get the chance to understand the business process in detail and make the decision on what to do with the employee. For instance, they are given the opportunity to study the business plan and will find out the benefits of doing so. 

🟨 BODY 2:
 On the other hand, there are many other methods to choose the right person. In addition, there are also other techniques, which can be used to help the candidate. For instance, they have a chance to 